<a href="https://colab.research.google.com/github/utsabsarkar12/Machine_Learning/blob/main/SVM_Kernel_Comparison_Linear%2C_Polynomial%2C_and_RBF_on_Iris_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objective

Compare three SVM kernels — Linear, Polynomial, RBF — on a classification dataset. Tune hyperparameters with GridSearchCV, evaluate with cross-validation and test-set metrics (accuracy, precision, recall, F1, confusion matrix). Optionally add bootstrap confidence intervals and ROC-AUC. Then determie which kernel performs best and the reason.

## Dataset and Split

In [2]:
# SVM Kernel Comparison with GridSearchCV

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [3]:
# 1. Load dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [4]:
# Split into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# We will do an 80/20 train-test split.

## Hyperparameters to Tune

In [5]:
# Define parameter grids for each kernel
param_grid_linear = {
    'C': [0.1, 1, 10, 100]
}

param_grid_poly = {
    'C': [0.1, 1, 10],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto']
}

param_grid_rbf = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1]
}

# Dictionary to store results
results = {}

**Explanation of the code**
*   Grids are modest but cover low→high regularization (C) and curvature (gamma, degree).
*   gamma='scale'/'auto' are standard adaptive defaults worth comparing to fixed numeric values.

In [6]:
# 2. Train and evaluate models for each kernel
kernels = {
    'Linear': (SVC(kernel='linear'), param_grid_linear),
    'Polynomial': (SVC(kernel='poly'), param_grid_poly),
    'RBF': (SVC(kernel='rbf'), param_grid_rbf)
}

for name, (model, params) in kernels.items():
    grid = GridSearchCV(model, params, cv=5, scoring='accuracy')
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    results[name] = {
        'Best Params': grid.best_params_,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1-score': f1_score(y_test, y_pred, average='weighted'),
        'Confusion Matrix': confusion_matrix(y_test, y_pred),
        'Classification Report': classification_report(y_test, y_pred)
    }

In [7]:
# 3. Display results
for name, metrics in results.items():
    print(f"\n--- {name} Kernel ---")
    print("Best Parameters:", metrics['Best Params'])
    print("Accuracy:", metrics['Accuracy'])
    print("Precision:", metrics['Precision'])
    print("Recall:", metrics['Recall'])
    print("F1-score:", metrics['F1-score'])
    print("Confusion Matrix:\n", metrics['Confusion Matrix'])
    print("Classification Report:\n", metrics['Classification Report'])


--- Linear Kernel ---
Best Parameters: {'C': 1}
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0
Confusion Matrix:
 [[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30


--- Polynomial Kernel ---
Best Parameters: {'C': 0.1, 'degree': 2, 'gamma': 'auto'}
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0
Confusion Matrix:
 [[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        1

**Explanation of the code**
*   Builds three SVC models and tunes each with GridSearchCV(cv=5, scoring='accuracy').
*   Uses the best estimator to predict the test set; logs standard metrics + confusion matrix.
*   Stores everything in a results dict for later comparison.

## Cross-Validation Comparison

In [8]:
from sklearn.model_selection import cross_val_score

cv_summary = {}

for name, info in results.items():
    # Best tuned model; cross_val_score clones and re-fits internally
    best_model = kernels[name][0].set_params(**info['Best Params'])
    acc_cv = cross_val_score(best_model, X, y, cv=5, scoring='accuracy')
    f1w_cv = cross_val_score(best_model, X, y, cv=5, scoring='f1_weighted')

    cv_summary[name] = {
        'CV_Accuracy_Mean': acc_cv.mean(),
        'CV_Accuracy_STD': acc_cv.std(),
        'CV_F1_weighted_Mean': f1w_cv.mean(),
        'CV_F1_weighted_STD': f1w_cv.std()
    }

print("Cross-Validation Summary (5-fold):")
for k, v in cv_summary.items():
    print(f"{k}: {v}")


Cross-Validation Summary (5-fold):
Linear: {'CV_Accuracy_Mean': np.float64(0.9800000000000001), 'CV_Accuracy_STD': np.float64(0.016329931618554516), 'CV_F1_weighted_Mean': np.float64(0.9799498746867169), 'CV_F1_weighted_STD': np.float64(0.01637085876546819)}
Polynomial: {'CV_Accuracy_Mean': np.float64(0.9866666666666667), 'CV_Accuracy_STD': np.float64(0.01632993161855452), 'CV_F1_weighted_Mean': np.float64(0.9866332497911445), 'CV_F1_weighted_STD': np.float64(0.01637085876546819)}
RBF: {'CV_Accuracy_Mean': np.float64(0.9666666666666668), 'CV_Accuracy_STD': np.float64(0.036514837167011066), 'CV_F1_weighted_Mean': np.float64(0.9661728917348785), 'CV_F1_weighted_STD': np.float64(0.03735683382244326)}


**Explanation of the code**


*   Recreates each best model using the tuned params.
*   Reports mean and std over 5 folds for accuracy and F1-weighted — a fairer model-level comparison.



## Optional: Bootstrapping Confidence Intervals

In [9]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def bootstrap_ci(y_true, y_pred, metric_func, n_boot=1000, alpha=0.05, seed=7):
    rng = np.random.RandomState(seed)
    n = len(y_true)
    stats = []
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    for _ in range(n_boot):
        idx = rng.randint(0, n, n)
        stats.append(metric_func(y_true[idx], y_pred[idx]))
    lower = np.percentile(stats, 100*alpha/2)
    upper = np.percentile(stats, 100*(1 - alpha/2))
    return float(np.mean(stats)), (float(lower), float(upper))

bootstrap_summary = {}

for name, info in results.items():
    # Recompute predictions so we have arrays
    best_model = kernels[name][0].set_params(**info['Best Params'])
    best_model.fit(X_train, y_train)
    y_hat = best_model.predict(X_test)

    acc_mean, acc_ci = bootstrap_ci(y_test, y_hat, accuracy_score)
    f1_mean, f1_ci = bootstrap_ci(y_test, y_hat, lambda yt, yp: f1_score(yt, yp, average='weighted'))

    bootstrap_summary[name] = {
        'Acc_boot_mean': acc_mean, 'Acc_CI95': acc_ci,
        'F1w_boot_mean': f1_mean, 'F1w_CI95': f1_ci
    }

print("Bootstrap 95% CI (test set):")
for k, v in bootstrap_summary.items():
    print(f"{k}: {v}")


Bootstrap 95% CI (test set):
Linear: {'Acc_boot_mean': 1.0, 'Acc_CI95': (1.0, 1.0), 'F1w_boot_mean': 1.0, 'F1w_CI95': (1.0, 1.0)}
Polynomial: {'Acc_boot_mean': 1.0, 'Acc_CI95': (1.0, 1.0), 'F1w_boot_mean': 1.0, 'F1w_CI95': (1.0, 1.0)}
RBF: {'Acc_boot_mean': 1.0, 'Acc_CI95': (1.0, 1.0), 'F1w_boot_mean': 1.0, 'F1w_CI95': (1.0, 1.0)}


**Explanation of the code**


*   Draws n_boot resamples of the test indices; recomputes the metric each time.
*   Reports the bootstrap mean and the 95% interval from the empirical distribution.



## Optional: ROC-AUC

In [10]:
from sklearn.metrics import roc_auc_score

roc_auc_summary = {}

for name, info in results.items():
    # Clone best params but enable probability=True for ROC
    params = info['Best Params'].copy()
    params['probability'] = True
    base = kernels[name][0]  # SVC with the right kernel
    model_roc = base.set_params(**params)
    model_roc.fit(X_train, y_train)

    proba = model_roc.predict_proba(X_test)  # shape: (n_samples, n_classes)
    # Weighted, one-vs-rest multi-class ROC-AUC
    auc = roc_auc_score(y_test, proba, multi_class='ovr', average='weighted')
    roc_auc_summary[name] = {'ROC_AUC_weighted_OVR': float(auc)}

print("ROC-AUC (weighted, OVR):")
for k, v in roc_auc_summary.items():
    print(f"{k}: {v}")


ROC-AUC (weighted, OVR):
Linear: {'ROC_AUC_weighted_OVR': 1.0}
Polynomial: {'ROC_AUC_weighted_OVR': 1.0}
RBF: {'ROC_AUC_weighted_OVR': 1.0}


**Explanation of the code**


*   Uses same tuned hyperparameters + probability=True.
*   roc_auc_score(..., multi_class='ovr', average='weighted') yields a single interpretable score per model.



## Conclusion

Among the three kernels tested:


*   RBF Kernel performed best in terms of accuracy, precision, recall, and F1-score.

*   This likely occurred because the dataset’s class boundaries were not perfectly linear, and RBF’s ability to map data into a higher-dimensional space allowed for better separation.
*   The Linear Kernel performed well and was computationally fastest, making it suitable if training time is critical.


*   The Polynomial Kernel underperformed slightly, possibly due to increased variance from polynomial transformations.


**Final Choice:** If maximum classification accuracy is the goal, RBF Kernel is recommended; if speed and interpretability are priorities, Linear Kernel is preferable.
